In [107]:
# %pip install scikit-learn

In [108]:
import numpy as np
from sklearn.preprocessing import RobustScaler, OneHotEncoder
import pickle as pk
import pandas as pd
from tensorflow import keras
import scipy
np.random.seed(0)

In [109]:
# with open("../data/pickles/data_slidingwindowFalse10_samplingrate20_df.p", 'rb') as f:
#   data = pk.load(f)
# dataframe = data['data']
# test_dataframe = dataframe[dataframe['pid'] == 'BU4707']
# dataframe = dataframe[(dataframe['pid'] != 'BK7610') & (dataframe['pid']!='BU4707')]

# test_dataframe.shape

In [110]:
## PID = DC6359
df = pd.read_csv("../data/DC6359_feature_engg_updated.csv")
df = df.sample(frac=1, random_state=1).reset_index()
df = df.drop(['Unnamed: 0'], axis=1) 
df.head()

,tac,x_mean,y_mean,z_mean,x_std,y_std,z_std,x_aad,y_aad,z_aad,...,y_skewness_fft,z_skewness_fft,x_kurtosis_fft,y_kurtosis_fft,z_kurtosis_fft,x_energy_fft,y_energy_fft,z_energy_fft,avg_result_accl_fft,sma_fft
0,0,-0.000522,-0.001690,-0.001607,0.012240,0.007164,0.010232,0.008288,0.005044,0.007790,...,0.382507,0.252335,-0.597043,-0.470893,-0.597490,0.015067,0.005144,0.010473,0.166114,0.260122
1,0,-0.004428,0.000969,-0.000996,0.009895,0.008558,0.012174,0.007525,0.006547,0.008964,...,1.111469,0.609624,-1.028063,0.992759,-0.071901,0.009792,0.007507,0.015176,0.171882,0.275326
2,0,-0.002430,0.003917,0.000383,0.018988,0.013274,0.018427,0.013756,0.009328,0.013305,...,0.019312,0.954931,-0.028103,-0.854662,0.259621,0.037391,0.017624,0.034380,0.282788,0.443865
3,0,-0.003570,0.003626,0.000294,0.026215,0.014554,0.020006,0.020313,0.010637,0.013874,...,0.746230,0.168985,0.328466,0.932649,-0.605917,0.069683,0.021186,0.040531,0.342740,0.535346
4,0,-0.003775,0.003096,-0.000126,0.023270,0.012352,0.014846,0.018516,0.010172,0.010447,...,0.383765,0.451049,-0.760536,-0.991467,-0.724613,0.054373,0.015269,0.022129,0.281318,0.442524


In [ ]:
# test_df = pd.read_csv("../data/BU4707_feature_engg_updated.csv")
# test_df = test_df.drop(['Unnamed: 0'], axis=1) 
# test_df.head()

In [132]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop(['tac'], axis=1), df.tac, test_size=0.25, random_state=1)

In [111]:
df = 

,tac,x_mean,y_mean,z_mean,x_std,y_std,z_std,x_aad,y_aad,z_aad,...,y_skewness_fft,z_skewness_fft,x_kurtosis_fft,y_kurtosis_fft,z_kurtosis_fft,x_energy_fft,y_energy_fft,z_energy_fft,avg_result_accl_fft,sma_fft
0,0,0.004578,0.007246,0.016726,0.003708,0.004323,0.004929,0.003109,0.003611,0.003963,...,0.562194,0.461567,0.196162,0.037270,-0.640511,0.001377,0.001879,0.002469,0.073160,0.118760
1,0,0.001759,0.010015,0.016626,0.013341,0.010943,0.010745,0.006699,0.006029,0.005773,...,-0.094469,0.643626,0.694876,-0.841908,0.796935,0.017846,0.012040,0.011658,0.196499,0.320673
2,0,0.002363,0.009744,0.013846,0.017084,0.013323,0.014038,0.010165,0.008480,0.009637,...,0.352616,0.360749,0.211095,-0.434230,-0.963782,0.029520,0.017757,0.019744,0.245845,0.396169
3,0,0.002435,0.008929,0.010908,0.011838,0.009810,0.012986,0.007481,0.007362,0.009902,...,0.203188,0.948351,1.158696,-0.739112,1.158584,0.014189,0.009638,0.016865,0.193401,0.310360
4,0,0.000526,0.009971,0.010272,0.005405,0.007611,0.010529,0.003908,0.005775,0.007740,...,0.685426,0.978296,-0.939616,-0.049094,0.752159,0.002948,0.005793,0.011107,0.132176,0.205270


In [112]:
# Test PID = BU4707
X_test = test_df.drop(['tac'], axis=1)
y_test = test_df.tac
X_test.shape, y_test.shape

((4490, 88), (4490,))

In [113]:
np.array(df.tac).reshape(-1, 1)

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]])

In [122]:
def create_dataset(X, y, time_steps=1, step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - time_steps, step):
        v = X.iloc[i:(i + time_steps)].values
        labels = y.iloc[i: i + time_steps]
        Xs.append(v)
        ys.append(scipy.stats.mode(labels)[0][0])
    return np.array(Xs), np.array(ys).reshape(-1, 1)

TIME_STEPS = 1
STEP = 1

X_train, y_train = create_dataset(
    # df[['x_skewness', 'y_skewness', 'z_skewness', 'x_kurtosis', 'y_kurtosis', 'z_kurtosis']],
    df.drop(['tac'], axis=1), 
    df.tac,
    TIME_STEPS,
    STEP
)


X_test, y_test = create_dataset(
    test_df.drop(['tac'], axis=1),
    test_df.tac,
    TIME_STEPS,
    STEP
)

# X_train =  df[['x_skewness', 'y_skewness', 'z_skewness', 'x_kurtosis', 'y_kurtosis', 'z_kurtosis']]
# y_train = df.tac

/var/folders/7y/kyw1v_8j0g1ckfb3g6x93q1m0000gn/T/ipykernel_23060/209021649.py:7: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  ys.append(scipy.stats.mode(labels)[0][0])
/var/folders/7y/kyw1v_8j0g1ckfb3g6x93q1m0000gn/T/ipykernel_23060/209021649.py:7: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or

In [123]:
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

enc = enc.fit(y_train)

y_train = enc.transform(y_train)
y_test = enc.transform(y_test)

/Users/sarah_prakriti_peters/miniconda3/envs/DL/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [124]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((6215, 1, 88), (6215, 2), (4489, 1, 88), (4489, 2))

In [125]:
model = keras.Sequential()
model.add(
    keras.layers.Bidirectional(
      keras.layers.LSTM(
          units=256,
          input_shape=[X_train.shape[1], X_train.shape[2]]
      )
    )
)
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dense(y_train.shape[1], activation='sigmoid'))

model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['acc']
)


In [126]:
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32, 
    validation_split=0.01,
    shuffle=False) 

Epoch 1/20
193/193 [==============================] - 4s 9ms/step - loss: 0.5036 - acc: 0.8280 - val_loss: 3.9208 - val_acc: 0.0000e+00
Epoch 2/20
193/193 [==============================] - 1s 5ms/step - loss: 0.4669 - acc: 0.8430 - val_loss: 4.1359 - val_acc: 0.0000e+00
Epoch 3/20
193/193 [==============================] - 1s 5ms/step - loss: 0.4722 - acc: 0.8711 - val_loss: 3.7827 - val_acc: 0.0000e+00
Epoch 4/20
193/193 [==============================] - 1s 5ms/step - loss: 0.4511 - acc: 0.8617 - val_loss: 3.8135 - val_acc: 0.0000e+00
Epoch 5/20
193/193 [==============================] - 1s 5ms/step - loss: 0.4898 - acc: 0.8201 - val_loss: 4.5379 - val_acc: 0.0000e+00
Epoch 6/20
193/193 [==============================] - 1s 5ms/step - loss: 0.5370 - acc: 0.8230 - val_loss: 5.3564 - val_acc: 0.0000e+00
Epoch 7/20
193/193 [==============================] - 1s 4ms/step - loss: 0.5799 - acc: 0.8453 - val_loss: 5.0480 - val_acc: 0.0000e+00
Epoch 8/20
193/193 [============================

In [127]:
# df[df['tac']==1].count(), df[df['tac']==0].count()

In [129]:
model.evaluate(X_test, y_test, verbose=1) 

141/141 [==============================] - 0s 1ms/step - loss: 1.3380 - acc: 0.7160


[1.3380383253097534, 0.715972363948822]

In [ ]:
# model.save("bidirectional_lstm.h5")

In [ ]:
model = keras.Sequential()
model.add(
    keras.layers.Bidirectional(
      keras.layers.LSTM(
          units=256,
          input_shape=[X_train.shape[1], X_train.shape[2]]
      )
    )
)
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dense(y_train.shape[1], activation='sigmoid'))

model.compile(
  loss='binary_crossentropy',
  optimizer=keras.optimizers.Adam(0.01),
  metrics=['acc']
)


history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32, 
    validation_split=0.01,
    shuffle=False) 


In [ ]:
model.evaluate(X_train, y_train)

## MLP Classifier

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
clf = MLPClassifier(solver='adam', shuffle=True, random_state=1)

In [ ]:
X_train.shape

In [ ]:
clf.fit(np.reshape(X_train, (224, 120)), y_train)
clf.get_params() 

In [ ]:
print('Accuracy ', accuracy_score(y_train, clf.predict(np.reshape(X_train, (224, 120)))))

## CNN 

In [ ]:
batch_size = 256
epochs = 20

flatten = keras.layers.Flatten()
conv_layer1 = keras.layers.Conv1D(filters = 64, kernel_size = 3, padding='SAME')
conv_layer2 = keras.layers.Conv1D(filters = 64, kernel_size = 3, padding='SAME')
dropout = keras.layers.Dropout(0.5)
max_pooling = keras.layers.MaxPool1D(pool_size=2)
fc_layer = keras.layers.Dense(units=128, activation = 'relu')
fc_layer2 = keras.layers.Dense(y_train.shape[1], activation = 'sigmoid')
base_model = keras.Sequential([
                                  conv_layer1,  
                                  conv_layer2, 
                                  dropout, 
                                  max_pooling, 
                                  flatten, 
                                  fc_layer, 
                                  fc_layer2
                                ])

base_model.compile(loss='binary_crossentropy', 
                   optimizer='adam',
                   metrics=['acc']) 

In [ ]:
history = base_model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32, 
    validation_split=0.01,
    shuffle=False) 

In [ ]:
base_model.evaluate(X_test, y_test)